<a href="https://colab.research.google.com/github/Emohamed152/my_learning/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
data=pd.read_csv("/content/enhanced_customer_support_data.csv")

In [17]:
data.shape

(13838, 12)

In [18]:
data.isnull().sum()

,0
Ticket_ID,0
Customer_Name,0
Customer_Email,0
Ticket_Subject,0
Ticket_Description,1
Issue_Category,1
Priority_Level,1
Ticket_Channel,1
Submission_Date,1
Resolution_Time_Hours,1


In [19]:
data.duplicated().sum()

np.int64(0)

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13838 entries, 0 to 13837
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Ticket_ID              13838 non-null  object 
 1   Customer_Name          13838 non-null  object 
 2   Customer_Email         13838 non-null  object 
 3   Ticket_Subject         13838 non-null  object 
 4   Ticket_Description     13837 non-null  object 
 5   Issue_Category         13837 non-null  object 
 6   Priority_Level         13837 non-null  object 
 7   Ticket_Channel         13837 non-null  object 
 8   Submission_Date        13837 non-null  object 
 9   Resolution_Time_Hours  13837 non-null  float64
 10  Assigned_Agent         13837 non-null  object 
 11  Satisfaction_Score     13837 non-null  float64
dtypes: float64(2), object(10)
memory usage: 1.3+ MB


Drop non important Columns (Customer_Name     ,  Customer_Email  ,       Assigned_Agent    ,  Satisfaction_Score ,  Submission_Date )

In [22]:
cols_to_drop = [
    "Customer_Name",
    "Customer_Email",
    "Assigned_Agent",
    "Submission_Date"
]

data = data.drop(columns=cols_to_drop)

KeyError: "['Customer_Name', 'Customer_Email', 'Assigned_Agent', 'Submission_Date'] not found in axis"

In [23]:
data.shape

(13838, 8)

In [24]:
data.head()

,Ticket_ID,Ticket_Subject,Ticket_Description,Issue_Category,Priority_Level,Ticket_Channel,Resolution_Time_Hours,Satisfaction_Score
0,TKT-100000,Hours of operation - Individual,"Hi Support, Where is your headquarters located...",General Inquiry,High,Web Form,43.0,5.0
1,TKT-100001,Data not syncing - Card,"Hi Support, The application crashes every time...",Technical,High,Chat,41.0,5.0
2,TKT-100002,2FA issues - Question,"Hi Support, How do I upgrade to the Enterprise...",Account,High,Web Form,7.0,5.0
3,TKT-100003,Login failed - Let,"Hi Support, The dashboard is not loading any d...",Technical,Low,Web Form,41.0,5.0
4,TKT-100004,Refund status - Attention,"Hi Support, I have been trying to update my pa...",Billing,Medium,Email,40.0,5.0


Encode Categorical Columns
(Issue_Category
Priority_Level
Ticket_Channel)

In [25]:
from sklearn.preprocessing import LabelEncoder

label_cols = ["Issue_Category", "Priority_Level", "Ticket_Channel"]
encoders = {}

for col in label_cols:
    enc = LabelEncoder()
    data[col + "_encoded"] = enc.fit_transform(data[col])
    encoders[col] = enc

In [26]:
data = data.drop(columns=label_cols)

Normalize Numerical Columns
(Resolution_Time_Hours
Satisfaction_Score)

In [27]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
num_cols = ["Resolution_Time_Hours", "Satisfaction_Score"]

data[num_cols] = scaler.fit_transform(data[num_cols])

mearge Subject + Description

In [28]:
data["full_text"] = data["Ticket_Subject"].astype(str) + " " + data["Ticket_Description"].astype(str)

In [29]:
data.head(15)

,Ticket_ID,Ticket_Subject,Ticket_Description,Resolution_Time_Hours,Satisfaction_Score,Issue_Category_encoded,Priority_Level_encoded,Ticket_Channel_encoded,full_text
0,TKT-100000,Hours of operation - Individual,"Hi Support, Where is your headquarters located...",0.106400,0.993314,3,1,2,"Hours of operation - Individual Hi Support, Wh..."
1,TKT-100001,Data not syncing - Card,"Hi Support, The application crashes every time...",0.049672,0.993314,4,1,0,"Data not syncing - Card Hi Support, The applic..."
2,TKT-100002,2FA issues - Question,"Hi Support, How do I upgrade to the Enterprise...",-0.914699,0.993314,0,1,2,"2FA issues - Question Hi Support, How do I upg..."
3,TKT-100003,Login failed - Let,"Hi Support, The dashboard is not loading any d...",0.049672,0.993314,4,2,2,"Login failed - Let Hi Support, The dashboard i..."
4,TKT-100004,Refund status - Attention,"Hi Support, I have been trying to update my pa...",0.021308,0.993314,1,3,1,"Refund status - Attention Hi Support, I have b..."
5,TKT-100005,Office location - National,"Hi Support, Where is your headquarters located...",-0.460877,0.217140,3,3,0,"Office location - National Hi Support, Where i..."
6,TKT-100006,Password reset - Body,"Hi Support, How do I upgrade to the Enterprise...",1.893322,-0.559033,0,3,0,"Password reset - Body Hi Support, How do I upg..."
7,TKT-100007,Payment failed - Win,"Hi Support, My subscription renewed automatica...",-0.347422,-2.111380,1,3,1,"Payment failed - Win Hi Support, My subscripti..."
8,TKT-100008,Login failed - Son,"Hi Support, I cannot log into my account even ...",0.730404,0.217140,4,1,2,"Login failed - Son Hi Support, I cannot log in..."
9,TKT-100009,Product question - Market,"Hi Support, Is there a roadmap for new feature...",2.006778,-0.559033,3,2,2,"Product question - Market Hi Support, Is there..."


Clean Text for NLP Models

In [30]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
stop = set(stopwords.words("english"))

def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"[^a-zA-Z0-9\s]", "", txt)
    txt = " ".join([w for w in txt.split() if w not in stop])
    return txt

data["clean_text"] = data["full_text"].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data.head(15)

,Ticket_ID,Ticket_Subject,Ticket_Description,Issue_Category,Priority_Level,Ticket_Channel,Submission_Date,full_text,clean_text,priority_encoded
0,TKT-100000,Hours of operation - Individual,"Hi Support, Where is your headquarters located...",General Inquiry,High,Web Form,2025-07-02,"Hours of operation - Individual Hi Support, Wh...",hours of operation individual hi support where...,2.0
1,TKT-100001,Data not syncing - Card,"Hi Support, The application crashes every time...",Technical,High,Chat,2025-06-28,"Data not syncing - Card Hi Support, The applic...",data not syncing card hi support the applicati...,2.0
2,TKT-100002,2FA issues - Question,"Hi Support, How do I upgrade to the Enterprise...",Account,High,Web Form,2025-02-05,"2FA issues - Question Hi Support, How do I upg...",2fa issues question hi support how do i upgrad...,2.0
3,TKT-100003,Login failed - Let,"Hi Support, The dashboard is not loading any d...",Technical,Low,Web Form,2025-03-20,"Login failed - Let Hi Support, The dashboard i...",login failed let hi support the dashboard is n...,0.0
4,TKT-100004,Refund status - Attention,"Hi Support, I have been trying to update my pa...",Billing,Medium,Email,2025-04-27,"Refund status - Attention Hi Support, I have b...",refund status attention hi support i have been...,1.0
5,TKT-100005,Office location - National,"Hi Support, Where is your headquarters located...",General Inquiry,Medium,Chat,2025-10-02,"Office location - National Hi Support, Where i...",office location national hi support where is y...,1.0
6,TKT-100006,Password reset - Body,"Hi Support, How do I upgrade to the Enterprise...",Account,Medium,Chat,2024-09-28,"Password reset - Body Hi Support, How do I upg...",password reset body hi support how do i upgrad...,1.0
7,TKT-100007,Payment failed - Win,"Hi Support, My subscription renewed automatica...",Billing,Medium,Email,2024-10-26,"Payment failed - Win Hi Support, My subscripti...",payment failed win hi support my subscription ...,1.0
8,TKT-100008,Login failed - Son,"Hi Support, I cannot log into my account even ...",Technical,High,Web Form,2025-04-29,"Login failed - Son Hi Support, I cannot log in...",login failed son hi support i cannot log into ...,2.0
9,TKT-100009,Product question - Market,"Hi Support, Is there a roadmap for new feature...",General Inquiry,Low,Web Form,2024-06-30,"Product question - Market Hi Support, Is there...",product question market hi support is there a ...,0.0


Drop Subject + Description and use full_text

In [31]:
data = data.drop(columns=["Ticket_Subject", "Ticket_Description"])

In [32]:
data.head(10)

,Ticket_ID,Resolution_Time_Hours,Satisfaction_Score,Issue_Category_encoded,Priority_Level_encoded,Ticket_Channel_encoded,full_text,clean_text
0,TKT-100000,0.106400,0.993314,3,1,2,"Hours of operation - Individual Hi Support, Wh...",hours operation individual hi support headquar...
1,TKT-100001,0.049672,0.993314,4,1,0,"Data not syncing - Card Hi Support, The applic...",data syncing card hi support application crash...
2,TKT-100002,-0.914699,0.993314,0,1,2,"2FA issues - Question Hi Support, How do I upg...",2fa issues question hi support upgrade enterpr...
3,TKT-100003,0.049672,0.993314,4,2,2,"Login failed - Let Hi Support, The dashboard i...",login failed let hi support dashboard loading ...
4,TKT-100004,0.021308,0.993314,1,3,1,"Refund status - Attention Hi Support, I have b...",refund status attention hi support trying upda...
5,TKT-100005,-0.460877,0.217140,3,3,0,"Office location - National Hi Support, Where i...",office location national hi support headquarte...
6,TKT-100006,1.893322,-0.559033,0,3,0,"Password reset - Body Hi Support, How do I upg...",password reset body hi support upgrade enterpr...
7,TKT-100007,-0.347422,-2.111380,1,3,1,"Payment failed - Win Hi Support, My subscripti...",payment failed win hi support subscription ren...
8,TKT-100008,0.730404,0.217140,4,1,2,"Login failed - Son Hi Support, I cannot log in...",login failed son hi support cannot log account...
9,TKT-100009,2.006778,-0.559033,3,2,2,"Product question - Market Hi Support, Is there...",product question market hi support roadmap new...
